# Running Queries on our Graph

In [1]:
from rdflib import Graph
g = Graph()
g.parse("../IFC Conversion/house.ttl")

g

<Graph identifier=N88fe0322006e40d4ba7e5d2f47a23c80 (<class 'rdflib.graph.Graph'>)>

## 1 - Using SPARQL

### Art. 22§2

The doors should not have a width below 1050mm.

In [2]:
ourQuery = """
PREFIX bot: <https://w3id.org/bot#>
PREFIX beo: <https://w3id.org/beo#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?s ?w
WHERE{ ?s rdf:type beo:Door .
    ?s ex:width ?w .
    FILTER (?w < 1050.0)
}
"""

qres = g.query(ourQuery)

print( f"Found {len(qres)} Doors with Width below 1050 mm: " )
for row in qres:
    print(f"{row.s} - door width = {row.w}")

Found 17 Doors with Width below 1050 mm: 
https://example.org#M_Single-Flush_0915x2032mm_398822 - door width = 915
https://example.org#M_Single-Flush_0915x2032mm_398909 - door width = 915
https://example.org#M_Single-Flush_0915x2032mm_399431 - door width = 915
https://example.org#M_Single-Flush_0915x2032mm_399501 - door width = 915
https://example.org#M_Single-Flush_0915x2032mm_399643 - door width = 915
https://example.org#M_Single-Flush_0915x2032mm_399925 - door width = 915
https://example.org#M_Single-Flush_0915x2032mm_400081 - door width = 915
https://example.org#M_Single-Flush_0915x2134mm_358185 - door width = 915
https://example.org#M_Single-Flush_0915x2134mm_358480 - door width = 915
https://example.org#M_Single-Flush_0915x2134mm_359949 - door width = 915
https://example.org#M_Single-Flush_0915x2134mm_377662 - door width = 915
https://example.org#M_Single-Flush_0915x2134mm_378185 - door width = 915
https://example.org#M_Single-Flush_0915x2134mm_378243 - door width = 915
https://e

### Art. 20§2

Stairs can have at most 17 risers.

In [3]:
ourQuery = """
# Check Riser Height
PREFIX ex: <https://example.org#>
PREFIX beo: <https://w3id.org/beo#>
select * where {
    ?s ?p beo:Stair .
    ?s ex:NumberOfRailings ?rl .
    ?s ex:NumberOfRiser ?ri .
    ?s ex:RiserHeight ?rh .
    ?s ex:TreadLength ?tl
    FILTER(?ri > 17)
} limit 100
"""

qres = g.query(ourQuery)

if len(qres) > 0:
    print( "Found these Stair(s) with more than 17 risers:" )
    for row in qres:
        print(f"{row.s} : Risers = {row.ri}")
else:
    print( "No stairs have more than 17 risers." )

Found these Stair(s) with more than 17 risers:
https://example.org#PrecastStair_Stair_383689 : Risers = 18


### Art. 20§3

The height of risers may be at most 18cm.

In [4]:
ourQuery = """
# Check Riser Height
PREFIX ex: <https://example.org#>
PREFIX beo: <https://w3id.org/beo#>
select * where {
    ?s ?p beo:Stair .
    ?s ex:NumberOfRailings ?r .
    ?s ex:RiserHeight ?rh .
    ?s ex:TreadLength ?tl
    FILTER(?rh > 180)
} limit 100
"""

qres = g.query(ourQuery)

if len(qres) > 0:
    print( "Found these Stairs too high risers:" )
    for row in qres:
        print(f"{row.s} : RH = {row.rh} TL = {row.tl}")
else:
    print( "All stairs have a small enough riser height." )

All stairs have a small enough riser height.


The length of treads should be at least 23cm.

In [5]:
ourQuery = """
# Check Riser Height
PREFIX ex: <https://example.org#>
PREFIX beo: <https://w3id.org/beo#>
select * where {
    ?s ?p beo:Stair .
    ?s ex:NumberOfRailings ?r .
    ?s ex:RiserHeight ?rh .
    ?s ex:TreadLength ?tl
    FILTER(?tl < 230)
} limit 100
"""

qres = g.query(ourQuery)

if len(qres) > 0:
    print( "Found these Stairs with too short tread lengths:" )
    for row in qres:
        print(f"{row.s} : RH = {row.rh} TL = {row.tl}")
else:
    print( "All stairs have a sufficient tread length." )

All stairs have a sufficient tread length.


### Art. 20§3
The stairs and the calculated stair formula: 2xRiser + 1x TreadLength

In [6]:
ourQuery = """
# Check Stair formula
PREFIX ex: <https://example.org#>
PREFIX beo: <https://w3id.org/beo#>
select * where {
    ?s ?p beo:Stair .
    ?s ex:NumberOfRailings ?r .
    ?s ex:RiserHeight ?rh .
    ?s ex:TreadLength ?tl
    BIND( (2 * ?rh + ?tl) AS ?StairFormula )
    FILTER((?StairFormula < 590) || (?StairFormula > 630))
} limit 100
"""

qres = g.query(ourQuery)

if len(qres) > 0:
    print( "Found these Stairs with non-compliant Stair Formula: " )
    for row in qres:
        print(f"{row.s} : 2xRH+TL = {row.StairFormula}")
else:
    print("All stairs comply with the Stair Formula")

Found these Stairs with non-compliant Stair Formula: 
https://example.org#PrecastStair_Stair_397996 : 2xRH+TL = 454


## 2 - Using SHACL

In [7]:
!pip install pyshacl

# See also https://shacl.org/playground/

### Default (empty) validation

In [8]:
from pyshacl import validate
from os import path

# Load our Model Data (TTL)
data_ttl_file = path.abspath("../IFC Conversion/house.ttl")

# Validate (but not shapes have been defined)
conforms, v_graph, v_text = validate(
    data_ttl_file, shacl_graph=None, inference='rdfs',
    serialize_report_graph=True)

# Output
print(conforms)
print(v_graph)
print(v_text)

True
b'@prefix ns1: <http://www.w3.org/ns/shacl#> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\n[] a ns1:ValidationReport ;\n    ns1:conforms true .\n\n'
Validation Report
Conforms: True



### Using SHACL shapes to validate properties

In [11]:
shape_query = '''
@prefix dash: <http://datashapes.org/dash#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <http://schema.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

@prefix beo: <https://w3id.org/beo#> .
@prefix ex: <https://example.org#> .

# -----------------------------
# STAIR SHAPES
# -----------------------------
ex:StairShape
    a sh:NodeShape ;
    sh:targetClass beo:Stair ;
    sh:property [
        sh:path ex:NumberOfRiser ;
        sh:maxExclusive 18 ;
        sh:datatype xsd:integer ;
        sh:message "Art.20§2 The amount of risers can be at most 17";
    ] ;
    sh:property [
        sh:path ex:RiserHeight ;
        sh:maxInclusive 180 ;
        sh:datatype xsd:integer ; # integer or double?
        sh:message "Art.20§3 Riser Height should be at most 18cm";
    ] ;
    sh:property [
        sh:path ex:TreadLength ;
        sh:minInclusive 230 ;
        sh:datatype xsd:integer ; # integer or double?
        sh:message "Art.20§3 Tread Length should be at least 23cm";
    ] .

# -----------------------------
# DOOR SHAPES
# -----------------------------
ex:DoorShape
    a sh:NodeShape ;
    sh:targetClass beo:Door ;
    sh:property [
        sh:path ex:height ;
        sh:datatype xsd:integer ;
        sh:minInclusive 2090 ;
        # sh:maxInclusive 1000 ;
        sh:message "Art.22§1 Doors should be at least 2.09m high." ;
    ] ;
    sh:property [
        sh:path ex:width ;
        sh:datatype xsd:integer ;
        sh:minInclusive 1050 ;
        # sh:maxInclusive 1000 ;
        sh:message "Art.22§2 Doors should be at least 1.05m wide." ;
    ] .

'''

conforms, v_graph, v_text = validate(
    data_ttl_file,
    shacl_graph=shape_query,
    inference='rdfs',
    serialize_report_graph=True
)
print(conforms)
print(v_graph)
print(v_text)

False
b'@prefix ex: <https://example.org#> .\n@prefix sh: <http://www.w3.org/ns/shacl#> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\n[] a sh:ValidationReport ;\n    sh:conforms false ;\n    sh:result [ a sh:ValidationResult ;\n            sh:focusNode ex:M_Single-Flush_0915x2134mm_358185 ;\n            sh:resultMessage "Art.22\xc2\xa72 Doors should be at least 1.05m wide." ;\n            sh:resultPath ex:width ;\n            sh:resultSeverity sh:Violation ;\n            sh:sourceConstraintComponent sh:MinInclusiveConstraintComponent ;\n            sh:sourceShape _:n3149c885713c4b3084d5b72046615bc2b5 ;\n            sh:value 915 ],\n        [ a sh:ValidationResult ;\n            sh:focusNode ex:M_Single-Flush_0915x2032mm_399925 ;\n            sh:resultMessage "Art.22\xc2\xa72 Doors should be at least 1.05m wide." ;\n            sh:resultPath ex:width ;\n            sh:resultSeverity sh:Violation ;\n            sh:sourceConstraintComponent sh:MinInclusiveConstraintComponent ;\

In [12]:
# What can we do with the v_graph?
shacl_results = Graph()
shacl_results.parse(v_graph)
shacl_results.serialize(destination="shacl_results.ttl")
shacl_results.serialize(destination="shacl_results.xml", format="xml")

ourQuery = """
PREFIX bot: <https://w3id.org/bot#>
PREFIX beo: <https://w3id.org/beo#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX sh: <http://www.w3.org/ns/shacl#>

SELECT *
WHERE{ ?s ?p ?o .
}
"""

qres = shacl_results.query(ourQuery)
print( f"Found violations against the regulations in our model." )
for row in qres:
    print(f"{row}")

Found violations against the regulations in our model.
(rdflib.term.URIRef('http://www.w3.org/ns/shacl#sourceConstraintComponent'), rdflib.term.BNode('n96e9e61a7620480e8dd0e9992a4a61b0b26'), rdflib.term.URIRef('http://www.w3.org/ns/shacl#MinInclusiveConstraintComponent'))
(rdflib.term.URIRef('http://www.w3.org/ns/shacl#value'), rdflib.term.BNode('n96e9e61a7620480e8dd0e9992a4a61b0b28'), rdflib.term.Literal('915', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('http://www.w3.org/ns/shacl#value'), rdflib.term.BNode('n96e9e61a7620480e8dd0e9992a4a61b0b21'), rdflib.term.Literal('915', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('http://www.w3.org/ns/shacl#sourceShape'), rdflib.term.BNode('n96e9e61a7620480e8dd0e9992a4a61b0b12'), rdflib.term.BNode('n96e9e61a7620480e8dd0e9992a4a61b0b8'))
(rdflib.term.URIRef('http://www.w3.org/ns/shacl#resultSeverity'), rdflib.term.BNode('n96e9e61a7620480e8dd0e9992a4